<a href="https://colab.research.google.com/github/Dheeraj64209/Fine-Tuned-Customer-Support-Chatbot-LoRA-QLoRA-/blob/master/groq_chat_analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import sys
import os
import re
import json
import time
import logging
import requests
!{sys.executable} -m pip install --upgrade openai requests
!pip install openai==1.107.2
!pip install requests==2.32.4




  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
Using cached requests-2.32.5-py3-none-any.whl (64 kB)
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
  Using cached requests-2.32.4-py3-none-any.whl.metadata (4.9 kB)
Using cached requests-2.32.4-py3-none-any.whl (64 kB)
  Attempting uninstall: requests
    Found existing installation: requests 2.32.5
    Uninstalling requests-2.32.5:
      Successfully uninstalled requests-2.32.5


In [ ]:
from typing import List, Dict, Any, Optional, Tuple

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("groq-demo")

# === Setup Groq Client (API Key + Base URL) ===
if "GROQ_API_KEY" not in os.environ:
    from getpass import getpass
    key = getpass("Enter GROQ API key (input hidden): ")
    os.environ["GROQ_API_KEY"] = key

GROQ_API_KEY = os.environ.get("GROQ_API_KEY")
GROQ_API_BASE = os.environ.get("GROQ_API_BASE", "https://api.groq.com/openai/v1")

In [ ]:
#this function gives the available model
def list_available_models():
    try:
        resp = client.models.list()
        data = getattr(resp, "data", resp)
        ids = [getattr(item, "id", None) for item in data]
        ids = [m for m in ids if m]
        print("Available models:", ids)
        return ids
    except Exception as e:
        logger.error("Failed to list models: %s", e)
        raise

def groq_chat_completion(messages: List[Dict[str, str]],
                         model: str = "llama-3.1-8b-instant",
                         functions: Optional[List[Dict]] = None,
                         function_call: Any = None,
                         temperature: float = 0.0,
                         timeout: int = 60):
    """
    Calls Groq API directly using requests
    """
    url = f"{GROQ_API_BASE}/chat/completions"
    headers = {
        "Authorization": f"Bearer {GROQ_API_KEY}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": model,
        "messages": messages,
        "temperature": temperature
    }
    if functions:
        payload["functions"] = functions
    if function_call:
        payload["function_call"] = function_call

    try:
        resp = requests.post(url, headers=headers, data=json.dumps(payload), timeout=timeout)
        resp.raise_for_status()
        return resp.json()
    except Exception as e:
        logger.error("Groq request failed: %s", e)
        raise

# === Regex extractors ===
EMAIL_RE = re.compile(r"([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)")
PHONE_RE = re.compile(r"(\+?\d[\d\-\s\(\)]{6,}\d)")
AGE_RE = re.compile(r"\b(age[:\s]*|i am |i'm )?([0-9]{1,3})\b", re.I)

def extract_emails(text: str) -> List[str]:
    return EMAIL_RE.findall(text)

def extract_phones(text: str) -> List[str]:
    return PHONE_RE.findall(text)

def extract_ages(text: str) -> List[int]:
    ages = []
    for m in AGE_RE.findall(text):
        num = m[-1]
        try:
            n = int(num)
            if 0 <= n <= 150:
                ages.append(n)
        except:
            pass
    return ages

def message_importance_score(msg: Dict[str,str]) -> float:
    score = 0.0
    text = msg.get("content", "")
    if extract_emails(text): score += 5.0
    if extract_phones(text): score += 4.0
    if extract_ages(text): score += 3.0
    keywords = ["signup", "register", "password", "urgent", "cancel", "subscribe", "payment", "billing"]
    for kw in keywords:
        if kw in text.lower():
            score += 1.0
    role = msg.get("role", "user")
    role_weight = {"system":1.5, "user":1.2, "assistant":1.0}.get(role,1.0)
    score *= role_weight
    score += min(len(text) / 200.0, 2.0)
    return score


In [ ]:
# === Conversation Manager with Versioned Memory + Rollback ===
class ConversationManager:
    def __init__(self, summarization_k: int = 3, keep_tail: int = 5):
        self.history: List[Dict[str, str]] = []
        self.turn_counter = 0
        self.summarization_k = summarization_k
        self.keep_tail = keep_tail
        self.snapshots: List[List[Dict[str,str]]] = []  # versioned memory

    def add_message(self, role: str, content: str) -> Tuple[bool, Optional[str]]:
        self.history.append({"role": role, "content": content})
        self.turn_counter += 1
        self.snapshots.append(self.history.copy())  # save snapshot
        if self.turn_counter % self.summarization_k == 0:
            summary = self._summarize()
            return True, summary
        return False, None

    def rollback(self, version: int) -> None:
        if 0 <= version < len(self.snapshots):
            self.history = self.snapshots[version].copy()
            self.turn_counter = len(self.history)
            logger.info(f"Rolled back to version {version}")
        else:
            logger.warning("Invalid rollback version")

    def _summarize(self) -> str:
        try:
            old_turns = self.history[:-self.keep_tail]
            old_text = "\n".join([f"{m['role']}: {m['content']}" for m in old_turns])
            response = groq_chat_completion(
                model="llama-3.1-8b-instant",
                messages=[
                    {"role": "system", "content": "You are a summarizer. Summarize the conversation concisely."},
                    {"role": "user", "content": old_text}
                ]
            )
            summary = response["choices"][0]["message"]["content"].strip()
            self.history = [{"role": "system", "content": f"SUMMARY: {summary}"}] + self.history[-self.keep_tail:]
            return summary
        except Exception as e:
            error_msg = f"[SUMMARIZATION FAILED: {e}]"
            self.history = [{"role": "system", "content": error_msg}] + self.history[-self.keep_tail:]
            return error_msg

    def get_truncated_by_turns(self, n: int) -> List[Dict[str, str]]:
        return self.history[-n:]

    def get_truncated_by_chars(self, n: int) -> List[Dict[str, str]]:
        result, total_chars = [], 0
        for m in reversed(self.history):
            if total_chars + len(m["content"]) > n:
                break
            result.insert(0, m)
            total_chars += len(m["content"])
        return result

    def show_history(self) -> List[Dict[str, str]]:
        return self.history

In [ ]:
# === Action-Item Extractor Schema ===
TASK_SCHEMA = {
    "type": "object",
    "properties": {
        "tasks": {
            "type": "array",
            "items": {"type": "string"},
            "description": "List of actionable tasks the user requested."
        }
    },
    "required": ["tasks"]
}

TASK_FUNCTION = {
    "name": "extract_tasks",
    "description": "Extract a list of action items from the conversation.",
    "parameters": TASK_SCHEMA
}

def extract_action_items(chat_text: str, model: str = "llama-3.1-8b-instant") -> List[str]:
    messages = [
        {"role":"system", "content":"You are an action-item extractor. Return tasks only."},
        {"role":"user", "content": f"Extract all tasks from this conversation:\n\n{chat_text}"}
    ]
    resp = groq_chat_completion(messages, model=model,
                                functions=[TASK_FUNCTION],
                                function_call={"name":"extract_tasks"})
    try:
        fn_call = resp["choices"][0]["message"].get("function_call", {})
        if fn_call and "arguments" in fn_call:
            args = json.loads(fn_call["arguments"])
            return args.get("tasks", [])
    except Exception as e:
        logger.error("Failed to parse tasks: %s", e)
    return []

In [ ]:
# === Demo Run: Multiple Sample Chats + Snapshots ===
if __name__ == "__main__":
    cm = ConversationManager(summarization_k=3, keep_tail=3)

    # Multiple sample chats
    all_chats = [
        [
            ("user", "Hi, my name is Alice Johnson. I live in Bengaluru."),
            ("assistant", "Nice to meet you, Alice! How can I help you today?"),
            ("user", "I want to sign up. My email is alice.j@example.com and phone is +91-9876543210."),
            ("assistant", "Got it — what's your age?"),
            ("user", "I am 29 years old."),
            ("user", "Please create an account and schedule a follow-up call tomorrow."),
        ],
        [
            ("user", "Hey, I'm Bob Smith. My email is bob.smith@example.com."),
            ("assistant", "Hello Bob! Do you need help with registration or billing?"),
            ("user", "I want to cancel my subscription and get a refund."),
        ],
        [
            ("user", "Hi, my name is Carla M, age 30, from London. Phone: 020-7946-0636."),
            ("assistant", "Hi Carla! What would you like me to do for you today?"),
            ("user", "Please update my payment method and send a confirmation email."),
        ]
    ]

    # Run each chat
    for idx, chat in enumerate(all_chats):
        print(f"\n=== Processing Chat #{idx+1} ===")
        for role, txt in chat:
            did, summary = cm.add_message(role, txt)
            print(f"Added ({role}): {txt}")
            if did:
                print("\n--- Summarization triggered ---")
                print(summary)
                print("--- end summary ---\n")

    # Show all snapshots
    print("\n📌 All Snapshot Versions:")
    for i, snap in enumerate(cm.snapshots):
        print(f"\nVersion {i}:")
        for turn in snap:
            print(f"  {turn['role']}: {turn['content']}")

    # Extract action items from final history
    all_text = "\n".join([f"{m['role']}: {m['content']}" for m in cm.show_history()])
    tasks = extract_action_items(all_text)
    print("\n✅ Extracted Tasks:", tasks)

    # Demo rollback
    rollback_version = 2
    print(f"\n--- 🔄 Rolling back to version {rollback_version} ---")
    cm.rollback(rollback_version)
    print("\n📌 History after rollback:")
    print(cm.show_history())


=== Processing Chat #1 ===
Added (user): Hi, my name is Alice Johnson. I live in Bengaluru.
Added (assistant): Nice to meet you, Alice! How can I help you today?
Added (user): I want to sign up. My email is alice.j@example.com and phone is +91-9876543210.

--- Summarization triggered ---
There is no conversation to summarize yet. What would you like to talk about? I can summarize our conversation at the end if you like.
--- end summary ---

Added (assistant): Got it — what's your age?
Added (user): I am 29 years old.
Added (user): Please create an account and schedule a follow-up call tomorrow.

--- Summarization triggered ---
SUMMARY: 
- Introduced yourself as Alice Johnson from Bengaluru.
- Asked for assistance.
- Provided your email address (alice.j@example.com) and phone number (+91-9876543210) to sign up.
--- end summary ---


=== Processing Chat #2 ===
Added (user): Hey, I'm Bob Smith. My email is bob.smith@example.com.
Added (assistant): Hello Bob! Do you need help with registr